# CVDDivergence Strategy Analysis

This notebook provides an in-depth analysis of the CVDDivergence strategy, including its logic, indicators, and backtest results.

## Setup

### Change Working directory to repository root

In [1]:
from constants import set_cwd_to_project_root, CONFIG_DIR
from main import generate_config

generate_config("binance")

set_cwd_to_project_root()

### Configure Freqtrade environment

In [2]:
from freqtrade.configuration import Configuration


config = Configuration.from_files([CONFIG_DIR])

# Define some constants
config["timeframe"] = "5m"
# Name of the strategy class
config["strategy"] = "CVDDivergence"

# Pair to analyze - Only use one pair here
pair = "BTC/USDT:USDT"

In [3]:
# Load data using values set above
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from constants import DATA_DIR

candles = load_pair_history(
    datadir=DATA_DIR,
    timeframe=config["timeframe"],
    pair=pair,
    data_format="feather",
    candle_type=CandleType.FUTURES,
)

# Confirm success
print(f"Loaded {len(candles)} rows of data for {pair} from {DATA_DIR}")
candles.head()

Loaded 77949 rows of data for BTC/USDT:USDT from F:\quantative-analysis\data


,date,open,high,low,close,volume
0,2024-01-01 00:00:00+00:00,42314.0,42437.2,42289.6,42437.1,1724.210
1,2024-01-01 00:05:00+00:00,42437.2,42474.1,42420.5,42446.8,994.003
2,2024-01-01 00:10:00+00:00,42446.8,42535.0,42445.2,42532.5,899.775
3,2024-01-01 00:15:00+00:00,42532.4,42603.2,42494.1,42494.1,1291.232
4,2024-01-01 00:20:00+00:00,42494.1,42533.1,42484.4,42509.4,418.463


## Load and run strategy

In [4]:
# Load strategy using values set above
from freqtrade.data.dataprovider import DataProvider
from freqtrade.resolvers import StrategyResolver

strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)
strategy.ft_bot_start()

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {"pair": pair})
df.tail()

,date,open,high,low,close,volume,day,delta,cvd,pivot_low,...,poc,atr,enter_tag,enter_long,enter_short,atr_stoploss_long,atr_stoploss_short,exit_tag,exit_long,exit_short
77944,2024-09-27 15:20:00+00:00,66342.0,66388.8,66270.0,66302.3,1052.536,2024-09-27,-72.0,1215.4,NaN,...,66235.173,159.052035,,NaN,NaN,65984.195931,66620.404069,POC_Upper,1.0,1.0
77945,2024-09-27 15:25:00+00:00,66302.3,66330.0,66222.3,66303.1,657.397,2024-09-27,27.7,1243.1,NaN,...,66235.173,155.384032,,NaN,NaN,65992.331936,66613.868064,POC_Upper,1.0,NaN
77946,2024-09-27 15:30:00+00:00,66303.2,66322.0,66231.0,66258.5,673.062,2024-09-27,-63.5,1179.6,NaN,...,66235.173,150.785173,,NaN,NaN,65956.929655,66560.070345,POC_Upper,1.0,NaN
77947,2024-09-27 15:35:00+00:00,66258.6,66276.0,66183.4,66200.0,610.517,2024-09-27,-75.1,1104.5,NaN,...,66235.173,146.629089,,NaN,NaN,65906.741822,66493.258178,POC_Upper,1.0,NaN
77948,2024-09-27 15:40:00+00:00,66200.0,66289.0,66158.4,66170.2,749.706,2024-09-27,-41.6,1062.9,NaN,...,66235.173,145.484154,,NaN,NaN,65879.231692,66461.168308,POC_Upper,1.0,NaN


### Display the trade details

In [5]:
# Report results
print(f"Generated {df['enter_long'].sum()} long entry signals")
print(f"Generated {df['enter_short'].sum()} short entry signals")
data = df.set_index("date", drop=False)
data.tail()

Generated 553.0 long entry signals
Generated 613.0 short entry signals


,date,open,high,low,close,volume,day,delta,cvd,pivot_low,...,poc,atr,enter_tag,enter_long,enter_short,atr_stoploss_long,atr_stoploss_short,exit_tag,exit_long,exit_short
date,,,,,,,,,,,,,,,,,,,,,
2024-09-27 15:20:00+00:00,2024-09-27 15:20:00+00:00,66342.0,66388.8,66270.0,66302.3,1052.536,2024-09-27,-72.0,1215.4,NaN,...,66235.173,159.052035,,NaN,NaN,65984.195931,66620.404069,POC_Upper,1.0,1.0
2024-09-27 15:25:00+00:00,2024-09-27 15:25:00+00:00,66302.3,66330.0,66222.3,66303.1,657.397,2024-09-27,27.7,1243.1,NaN,...,66235.173,155.384032,,NaN,NaN,65992.331936,66613.868064,POC_Upper,1.0,NaN
2024-09-27 15:30:00+00:00,2024-09-27 15:30:00+00:00,66303.2,66322.0,66231.0,66258.5,673.062,2024-09-27,-63.5,1179.6,NaN,...,66235.173,150.785173,,NaN,NaN,65956.929655,66560.070345,POC_Upper,1.0,NaN
2024-09-27 15:35:00+00:00,2024-09-27 15:35:00+00:00,66258.6,66276.0,66183.4,66200.0,610.517,2024-09-27,-75.1,1104.5,NaN,...,66235.173,146.629089,,NaN,NaN,65906.741822,66493.258178,POC_Upper,1.0,NaN
2024-09-27 15:40:00+00:00,2024-09-27 15:40:00+00:00,66200.0,66289.0,66158.4,66170.2,749.706,2024-09-27,-41.6,1062.9,NaN,...,66235.173,145.484154,,NaN,NaN,65879.231692,66461.168308,POC_Upper,1.0,NaN


## Load backtest results

In [6]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
from constants import BACKTEST_RESULTS_DIR

# Load backtest statistics
stats = load_backtest_stats(BACKTEST_RESULTS_DIR)

strategy = "CVDDivergence"
# Example usages:
print("Results per pair:")
print(stats["strategy"][strategy]["results_per_pair"])
print("\nPairlist:")
print(stats["strategy"][strategy]["pairlist"])
print(f"\nMarket change: {stats['strategy'][strategy]['market_change']}")
print(f"Drawdown start: {stats['strategy'][strategy]['drawdown_start']}")
print(f"Drawdown end: {stats['strategy'][strategy]['drawdown_end']}")

# Load backtested trades as dataframe
trades = load_backtest_data(BACKTEST_RESULTS_DIR)

if len(trades) > 0:
    print("\nExit reasons per pair:")
    print(trades.groupby("pair")["exit_reason"].value_counts())

Results per pair:
[{'key': 'BTC/USDT:USDT', 'trades': 137, 'profit_mean': 0.015837187457887356, 'profit_mean_pct': 1.58, 'profit_sum': 2.169694681730568, 'profit_sum_pct': 216.97, 'profit_total_abs': 5870.28207402, 'profit_total': 5.87028207402, 'profit_total_pct': 587.03, 'duration_avg': '0:58:00', 'wins': 85, 'draws': 0, 'losses': 52, 'winrate': 0.6204379562043796}, {'key': 'TOTAL', 'trades': 137, 'profit_mean': 0.015837187457887356, 'profit_mean_pct': 1.58, 'profit_sum': 2.169694681730568, 'profit_sum_pct': 216.97, 'profit_total_abs': 5870.28207402, 'profit_total': 5.87028207402, 'profit_total_pct': 587.03, 'duration_avg': '0:58:00', 'wins': 85, 'draws': 0, 'losses': 52, 'winrate': 0.6204379562043796}]

Pairlist:
['BTC/USDT:USDT']

Market change: 0.4869300682657392
Drawdown start: 2024-08-31 22:15:00
Drawdown end: 2024-09-21 23:40:00

Exit reasons per pair:
pair           exit_reason
BTC/USDT:USDT  VWAP_Lower     78
               Bull_Div       20
               stop_loss      20
 

## Plotting daily profit / equity line

In [7]:
import pandas as pd
import plotly.express as px

strategy_stats = stats["strategy"][strategy]

df = pd.DataFrame(columns=["dates", "equity"], data=strategy_stats["daily_profit"])
df["equity_daily"] = df["equity"].cumsum()

fig = px.line(df, x="dates", y="equity_daily", title="CVDDivergence Strategy - Equity Curve")
fig.show()

## Plot results

In [8]:
from freqtrade.plot.plotting import generate_candlestick_graph

from_date = "2024-09-01"
to_date = "2024-09-27"

# Filter trades to one pair
trades_filtered = trades.loc[(trades["pair"] == pair) & (trades["open_date"] >= pd.to_datetime(from_date, utc=True)) & (trades["close_date"] <= pd.to_datetime(to_date, utc=True))]

# Filter data for a specific date range
data_filtered = data[from_date:to_date].drop(columns=["exit_short", "exit_long"])

# Generate candlestick graph with indicators for trade entry and losses
graph = generate_candlestick_graph(
    pair=pair,
    data=data_filtered,
    trades=trades_filtered,
    indicators1=["vwap", "vwap_lower_2", "vwap_upper_2", "poc"],
    indicators2=["cvd_ma"]
)

# Render graph in a separate window
graph.show(renderer="browser")

F:\freqtrade\freqtrade\plot\plotting.py:263: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
trades_filtered

,pair,stake_amount,max_stake_amount,amount,open_date,close_date,open_rate,close_rate,fee_open,fee_close,...,min_rate,max_rate,is_open,enter_tag,leverage,is_short,open_timestamp,close_timestamp,orders,desc
120,BTC/USDT:USDT,8538.949487,143529.2632,2.476,2024-09-03 21:40:00+00:00,2024-09-03 22:05:00+00:00,57968.2,58125.9,0.0005,0.0005,...,57941.0,58157.0,False,Bull_POC,16.808773,False,1725399600000,1725401100000,"[{'amount': 2.476, 'safe_price': 57968.2, 'ft_...","2.89%, Bull_POC, Bear_Div, 25 min"
121,BTC/USDT:USDT,8782.384814,136397.0569,2.353,2024-09-03 22:10:00+00:00,2024-09-03 23:00:00+00:00,57967.3,57780.7,0.0005,0.0005,...,57755.9,58043.9,False,Bull_POC,15.530754,False,1725401400000,1725404400000,"[{'amount': 2.353, 'safe_price': 57967.3, 'ft_...","-6.55%, Bull_POC, stop_loss, 50 min"
122,BTC/USDT:USDT,8212.752011,148641.5384,2.572,2024-09-03 23:05:00+00:00,2024-09-03 23:25:00+00:00,57792.2,57632.6,0.0005,0.0005,...,57631.0,57796.4,False,Bull_POC,18.098871,False,1725404700000,1725405900000,"[{'amount': 2.572, 'safe_price': 57792.2, 'ft_...","-6.80%, Bull_POC, stop_loss, 20 min"
123,BTC/USDT:USDT,7660.608929,135951.5290,2.359,2024-09-03 23:30:00+00:00,2024-09-03 23:50:00+00:00,57631.0,57468.7,0.0005,0.0005,...,57320.0,57725.8,False,Bull_POC,17.746831,False,1725406200000,1725407400000,"[{'amount': 2.359, 'safe_price': 57631.0, 'ft_...","-6.77%, Bull_POC, stop_loss, 20 min"
124,BTC/USDT:USDT,7144.954589,103486.8778,1.802,2024-09-03 23:55:00+00:00,2024-09-04 00:05:00+00:00,57428.9,57632.6,0.0005,0.0005,...,57384.6,57737.7,False,Bull_POC,14.483910,False,1725407700000,1725408300000,"[{'amount': 1.802, 'safe_price': 57428.9, 'ft_...","3.65%, Bull_POC, VWAP_Lower, 10 min"
125,BTC/USDT:USDT,7406.091939,123572.8575,2.297,2024-09-07 23:10:00+00:00,2024-09-08 00:05:00+00:00,53797.5,54093.9,0.0005,0.0005,...,53797.5,54200.0,False,Bull_POC,16.685299,False,1725750600000,1725753900000,"[{'amount': 2.297, 'safe_price': 53797.5, 'ft_...","7.45%, Bull_POC, VWAP_Lower, 55 min"
126,BTC/USDT:USDT,7951.543163,131603.0585,2.291,2024-09-11 23:00:00+00:00,2024-09-12 00:05:00+00:00,57443.5,57343.8,0.0005,0.0005,...,57260.1,57479.1,False,Bear_POC,16.550631,True,1726095600000,1726099500000,"[{'amount': 2.291, 'safe_price': 57443.5, 'ft_...","1.19%, Bear_POC, VWAP_Lower, 65 min"
127,BTC/USDT:USDT,8044.960752,99952.0500,1.650,2024-09-13 23:00:00+00:00,2024-09-14 00:05:00+00:00,60577.0,60509.4,0.0005,0.0005,...,60305.5,60590.0,False,Bear_POC,12.424181,True,1726268400000,1726272300000,"[{'amount': 1.65, 'safe_price': 60577.0, 'ft_o...","0.04%, Bear_POC, VWAP_Lower, 65 min"
128,BTC/USDT:USDT,8047.848414,197770.1130,3.294,2024-09-14 22:05:00+00:00,2024-09-14 22:20:00+00:00,60039.5,59922.1,0.0005,0.0005,...,59908.2,60042.5,False,Bear_POC,24.574284,True,1726351500000,1726352400000,"[{'amount': 3.294, 'safe_price': 60039.5, 'ft_...","2.35%, Bear_POC, Bull_Div, 15 min"
129,BTC/USDT:USDT,8235.803715,216716.3880,3.612,2024-09-14 22:35:00+00:00,2024-09-15 00:05:00+00:00,59999.0,59980.0,0.0005,0.0005,...,59900.6,60096.1,False,Bear_POC,26.313933,True,1726353300000,1726358700000,"[{'amount': 3.612, 'safe_price': 59999.0, 'ft_...","-1.81%, Bear_POC, VWAP_Lower, 90 min"


## Conclusion

This notebook provides an in-depth analysis of the CVDDivergence strategy. Based on the results, we can draw the following conclusions:

1. Trade Performance: [Summarize the overall performance, profitability, and drawdown]
2. Entry/Exit Signals: [Discuss the effectiveness of the entry and exit signals]
3. Pair Performance: [Highlight any pairs that performed particularly well or poorly]
4. Trade Parallelism: [Comment on the strategy's ability to manage multiple concurrent trades]
5. Profit Distribution: [Analyze the distribution of profits, noting any skewness or outliers]

Areas for improvement:
1. [Suggestion 1]
2. [Suggestion 2]
3. [Suggestion 3]

Next steps:
1. [Step 1]
2. [Step 2]
3. [Step 3]

Remember to thoroughly test any changes before using this strategy in live trading.